In [1]:
# Mount google drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Imports.
import os
import math as mt
import numpy as np
import cv2
from cv2 import (VideoCapture as VC, CAP_PROP_FRAME_COUNT as n_frames, 
                 VideoWriter as CW, CAP_PROP_FPS as n_fps)
from cv2.dnn import readNetFromDarknet as DarkNet

Mounted at /content/drive


In [3]:
# Change directory.
os.chdir('/content/drive/My Drive/MA305_Project/transfer_video')

# Inputs.
input_video = 'videos/video_nov_2.mp4'
output_video = 'output/video_nov_2.mp4'
weights_file = 'coco.weights'
cfg_file = 'coco.cfg'

In [4]:
# Load the labels from COCO.
with open('coco.csv', 'r') as lines:
  coco_labels = lines.readlines()[0].split(',')
print(coco_labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'bear', 'zebra', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'spoon', 'bowl', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush\n']


In [5]:
# Use OpenCV to load the video and obtain the number of frames.
video = VC(input_video)
number_of_frames = int(video.get(n_frames))
fps = video.get(n_fps)

In [6]:
net = DarkNet(cfg_file, weights_file)

In [7]:
# Function
## UPDATE ##
def image_process(frame, net, labels, args_confidence=0.5, 
                  args_threshold=0.3):
    
		# Get colors.
		np.random.seed(42)
		colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

		# Obtain the shape of the image.
		(H, W) = frame.shape[:2]

		# Construct a blob from the input frame and then perform a forward
		# pass of the YOLO object detector, giving us our bounding boxes
		# and associated probabilities
		blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
																	swapRB=True, crop=False)
		net.setInput(blob)
		##################

		# initialize our lists of detected bounding boxes, confidences,
		# and class IDs, respectively
		boxes = []
		confidences = []
		classIDs = []

   	# loop over each of the layer outputs
		output = net.forward(['yolo_106'])[0]
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]
			# filter out weak predictions by ensuring the detected
			# probability is greater than the minimum probability
			if confidence > args_confidence:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")
				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))
				# update our list of bounding box coordinates,
				# confidences, and class IDs
				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)

		# Apply non-maxima suppression to suppress weak, overlapping
		# bounding boxes.
		idxs = cv2.dnn.NMSBoxes(boxes, confidences, args_confidence, args_threshold)
    
		# Ensure at least one detection exists.
		labs = []
		if len(idxs) > 0:

			# Loop over the indexes we are keeping.
			for i in idxs.flatten():

				# Extract the bounding box coordinates.
				(x, y) = (boxes[i][0], boxes[i][1])
				(w, h) = (boxes[i][2], boxes[i][3])
				
				# Draw a bounding box rectangle and label on the frame.
				color = [int(c) for c in colors[classIDs[i]]]
				cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
				idx = classIDs[i]
				labs.append(idx)
				label = labels[idx]

				text = "{}: {:.4f}".format(label, confidences[i])
				cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
		# Frame
		return frame, labs

In [ ]:
# Create a new video.
_, first_frame = video.read()
video_shape = first_frame.shape
new_video = CW(output_video, 1196444237, mt.floor(fps), 
              (video_shape[1], video_shape[0]), isColor = True)

# Import the DarkNet model, and load the pre-trained weights.
net = DarkNet(cfg_file, weights_file)

classes = []
count = 0
n = int(fps)
save_frames = False
while video.isOpened():

    # Print the current frame number, and progress. 
    progress = round(count*100/number_of_frames, 2)
    print('Frame count: ', count, ', Progress: ', progress, '%')
    if progress == 100:
      break

    # Read the next frame.
    _, frame = video.read()

    # Process the current frame using the image_process function.
    # This returns a frame with the identified objects and their 
    # respective rectangles. It also returns the indexes of the classes
    # that have been identified in the image.
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE) 
    frame, idxs = image_process(frame, net, coco_labels)
    classes.append(idxs)

    # Save frame to the new video.
    # new_video.write(frame)

    # Save frames (only if more than 5 objects were identified).
    if save_frames and len(idxs) > 2:
      print('Saving frame: ', count)
      cv2.imwrite('output/frames/frame{:d}.jpg'.format(count), frame)

    # Set the count of the next frma to be processed.
    count += n
    video.set(1, count)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Frame count:  11256 , Progress:  14.51 %
Frame count:  11268 , Progress:  14.52 %
Frame count:  11280 , Progress:  14.54 %
Frame count:  11292 , Progress:  14.55 %
Frame count:  11304 , Progress:  14.57 %
Frame count:  11316 , Progress:  14.58 %
Frame count:  11328 , Progress:  14.6 %
Frame count:  11340 , Progress:  14.61 %
Frame count:  11352 , Progress:  14.63 %
Frame count:  11364 , Progress:  14.65 %
Frame count:  11376 , Progress:  14.66 %
Frame count:  11388 , Progress:  14.68 %
Frame count:  11400 , Progress:  14.69 %
Frame count:  11412 , Progress:  14.71 %
Frame count:  11424 , Progress:  14.72 %
Frame count:  11436 , Progress:  14.74 %
Frame count:  11448 , Progress:  14.75 %
Frame count:  11460 , Progress:  14.77 %
Frame count:  11472 , Progress:  14.79 %
Frame count:  11484 , Progress:  14.8 %
Frame count:  11496 , Progress:  14.82 %
Frame count:  11508 , Progress:  14.83 %
Frame count:  11520 , 

In [ ]:
np_array_classes = np.array(classes)

In [ ]:
np_array_classes.shape
np.save('time_2_10_35.npy', np_array_classes)

In [ ]:
np.load('time_2_10_35.npy', allow_pickle=True).shape